## Some background
- The complete code is: the train number (001-120) + the coach number (there are 6 coaches in a train) + the door number (4 doors in a coach). However, the dataset only has train number (column G), so the train number is enough.
- Note: even though the DIN code is present in the dataset and also in the pdfs provided, there is no relationship between them (Iguess the pdfs provided are not related to this specific dataset in general)
- After having analyzed the old dataset, and after all the questions I asked on Teams, the supervisor sent me the new dataset (componentwise). You can discard the old one and work on the componentwise!

### OLD:

In [ ]:
import pandas as pd
datapath = "data/raw/doorwise_dataset.xlsx"

cols_en = [
"Owner Name_2 (TPL)",
"Vehicle family (TPL)",
"EFA No (MEL)",
"DIN code UPG (H)",
"Assembly (DIN code) (MEL)",
"Vehicle type (TPL)",
"Multiple unit / EW (TPL)",
"Functional location (TPL)",
"Fault start date (MEL)",
"Short text (MEL)",
"Long text (MEL)",
"Message number (MEL)",
"Material (CO)",
"Material reference (MAT)",
"Completion date (MEL)",
"Number of messages",
"Number of orders",
"Direct manufacturing costs ACTUAL [CHF] ",
"Material quantity ACTUAL",
"Actual expenditure [h]"
]

cols_interst = [
"Multiple unit / EW (TPL)",
"Fault start date (MEL)",
"Short text (MEL)",
"Long text (MEL)",
"Message number (MEL)"
]

In [27]:
df = pd.read_excel(datapath, sheet_name="Sheet1")
df.columns = cols_en
df = df[cols_interst]

# Column G of the original dataset is the train number. Actually, only the digits after the 94 85 0511.
# Hence I'm going to remove the prefix (first check if the prefix doesn't change, maybe it has a meaning)
df.rename(columns={"Multiple unit / EW (TPL)": "train_number",
                   "Fault start date (MEL)" : "fault_start_date",
                   "Short text (MEL)" : "problem",
                   "Long text (MEL)" : "maintenance_log",
                   "Message number (MEL)" : "log_number"}, inplace=True)

In [28]:
tmp = None
for elem in df["train_number"]:
    e = elem[:10]
    if tmp == None or tmp != e:
        tmp = e
        print(tmp)

# Ok it is always the same. Remove the prefix:
df["train_number"]= df["train_number"].str[10:]

94 85 0511


In [29]:
df.head()

,train_number,fault_start_date,problem,maintenance_log,log_number
0,108-8,2020-10-28,Türe 4 MW2 schliesst nicht,* 28.10.2020 12:13:57 CET Marco Neukom (E5099...,125621058
1,113-8,2019-06-05,Tür summer Defekt,* 05.06.2019 08:39:49 CET Arijanit Ibrahimi (...,123843777
2,116-1,2018-06-24,-INT: Porte d'accès.,* 24.06.2018 22:32:06 CET Jean-Marc Robin (U1...,122600461
3,116-1,2019-11-18,ET 2 abgetrennt,* 18.11.2019 01:53:36 CET C. Hosmann (U118232...,124474801
4,116-1,2019-12-02,-Marchepied MT22,* 02.12.2019 19:04:04 CET Julien Roffat (U210...,124529054


## New dataset (componentwise)

In [2]:
import pandas as pd
datapath = "data/raw/componentwise_dataset.xlsx"
df = pd.read_excel(datapath, sheet_name="Export")

cols_en = [
"Fault start date (MEL)",
"Short text (MEL)",
"Long text (MEL)",
"Message number (MEL)",
"Vehicle family (TPL)",
"Vehicle type (TPL)",
"Material (CO)",
"Material reference (MAT)",
"Material quantity ACTUAL",
"Material / unit of measure (CO)",
"WF screen symptom (MEL)",
"WF-cause-sys-1 (MEL) (H)",
"WF-cause-sys-2 (MEL) (H)",
"WF-Cause-Sys-3 (MEL) (H)",
"WF-Cause-Sys-4 (MEL) (H)",
"WF-Cause-Sys-5 (MEL) (H)",
"WF-cause-sys-6 (MEL) (H)",
"WF cause failure type (MEL)",
"WF-cause-fix.mass (1-3) (MEL)",
"Multiple unit / EW (TPL)",
"Functional location (TPL)",
"DIN code (H)",
"Design workstation (location) (MEL)"
]

cols_interst = [
"Fault start date (MEL)",
"Short text (MEL)",
"Long text (MEL)",
"Message number (MEL)",
"WF-cause-sys-1 (MEL) (H)",
"WF-cause-sys-2 (MEL) (H)",
"WF-Cause-Sys-3 (MEL) (H)",
"WF-Cause-Sys-4 (MEL) (H)",
"Multiple unit / EW (TPL)"
]


In [3]:
# At the end there are some unnecessary inhomogenous rows. I filter out them
# Filter out rows from index 1595 (inclusive) onwards
df = df.loc[:1594]

In [4]:
for i, col in enumerate(df.columns):
    print(i, col)

0 Störungsbeginn Datum (MEL)
1 Kurztext (MEL)
2 Langtext (MEL)
3 Meldungsnummer (MEL)
4 Fz-Familie (TPL)
5 Fz-Typ (TPL)
6 Material (CO)
7 Material Bez (MAT)
8 Materialmenge IST
9 Material / Mengeneinheit (CO)
10 WF-Bild-Symptom (MEL)
11 WF-Ursache-Sys-1 (MEL) (H)
12 WF-Ursache-Sys-2 (MEL) (H)
13 WF-Ursache-Sys-3 (MEL) (H)
14 WF-Ursache-Sys-4 (MEL) (H)
15 WF-Ursache-Sys-5 (MEL) (H)
16 WF-Ursache-Sys-6 (MEL) (H)
17 WF-Ursache-Ausfallart (MEL)
18 WF-Ursache-Beheb.mass (1-3) (MEL)
19 Triebzug / EW (TPL)
20 Technischer Platz (TPL)
21 DIN-Code (H)
22 Ausf. Arbeitsplatz (Standort) (MEL)


In [5]:
df.columns = cols_en
df = df[cols_interst]

# Column G of the original dataset is the train number. Actually, only the digits after the 94 85 0511.
# Hence I'm going to remove the prefix (first check if the prefix doesn't change, maybe it has a meaning)
df.rename(columns={"Multiple unit / EW (TPL)": "train_number",
                   "Fault start date (MEL)" : "fault_start_date",
                   "Short text (MEL)" : "problem",
                   "Long text (MEL)" : "maintenance_log",
                   "Message number (MEL)" : "log_number",
                   "WF-cause-sys-1 (MEL) (H)" : "cause_sys1",
                   "WF-cause-sys-2 (MEL) (H)" : "cause_sys2",
                   "WF-Cause-Sys-3 (MEL) (H)" : "cause_sys3",
                   "WF-Cause-Sys-4 (MEL) (H)" : "cause_sys4"}, inplace=True)

In [6]:
# Check datatypes and convert them
print(df.dtypes)
print()

df["fault_start_date"] = pd.to_datetime(df["fault_start_date"], format="%Y-%m-%d %H:%M:%S")
df["probelm"] = df["problem"].astype(str)
df["maintenance_log"] = df["maintenance_log"].astype(str)
df["log_number"] = df["log_number"].astype(int)
df["cause_sys1"] = df["cause_sys1"].astype(str)
df["cause_sys2"] = df["cause_sys2"].astype(str)
df["cause_sys3"] = df["cause_sys3"].astype(str)
df["cause_sys4"] = df["cause_sys4"].astype(str)
df["train_number"] = df["train_number"].astype(str)


print(df.dtypes)

fault_start_date     object
problem              object
maintenance_log      object
log_number          float64
cause_sys1           object
cause_sys2           object
cause_sys3           object
cause_sys4           object
train_number         object
dtype: object

fault_start_date    datetime64[ns]
problem                     object
maintenance_log             object
log_number                   int64
cause_sys1                  object
cause_sys2                  object
cause_sys3                  object
cause_sys4                  object
train_number                object
probelm                     object
dtype: object


In [7]:
tmp = None
for elem in df["train_number"]:
    e = str(elem)[:10]
    if tmp == None or tmp != e:
        tmp = e
        print(tmp)

# Ok it is always the same. Remove the prefix:
df["train_number"]= df["train_number"].str[10:]

# Reorder columns
df = df[["train_number", "problem", "log_number", "maintenance_log", "fault_start_date", 
         "cause_sys1", "cause_sys2", "cause_sys3", "cause_sys4"]]

94 85 0511


In [8]:
df.head()

,train_number,problem,log_number,maintenance_log,fault_start_date,cause_sys1,cause_sys2,cause_sys3,cause_sys4
0,004-9,+ET-Notöffnug-Scheibe 1 fehlt,131511699,* 26.09.2024 15:07:37 CET (U241403) * System ...,2024-09-26,090 - Einstiegssysteme,010 - Einstiegstüre,050 - Türnotöffnung,010 - Einschlagscheibe Türnotöffnung
1,026-2,ET vitre ouv.secours 2 cassé,131506992,* 25.09.2024 12:01:18 CET (U244539) * Système...,2024-09-25,090 - Einstiegssysteme,010 - Einstiegstüre,050 - Türnotöffnung,010 - Einschlagscheibe Türnotöffnung
2,058-5,Sch.Tritt Zahnriemen 1 abgenutzt,131503871,* 25.09.2024 00:07:35 CET (U244162) * System ...,2024-09-25,090 - Einstiegssysteme,030 - Schiebetritt,060 - Trittmechanik,040 - Spindel / Zahnriemen
3,004-9,-ET 3 schliesst nicht,131500313,* 24.09.2024 06:00:37 CET (U222454) Tel. +41 ...,2024-09-24,090 - Einstiegssysteme,010 - Einstiegstüre,nan,nan
4,018-9,-ET calculateur porte 4 bloqué,131489381,* 20.09.2024 22:58:01 CET (U207064) Tél. +41 ...,2024-09-20,090 - Einstiegssysteme,010 - Einstiegstüre,020 - Türsteuerung lokal,010 - Türrechner


In [9]:
# Save interim dataframe
df.to_csv("data/interim/componentwise_df.csv", index=False)

## Component xlsx

In [106]:
# There are 2 sheets. I don't know the difference. Need to ask to supervisor
import pandas as pd
datapath = "data/raw/component.xlsx"
df1 = pd.read_excel(datapath, sheet_name="BBA Nummern im WmP-Baum")
df2 = pd.read_excel(datapath, sheet_name="V10")

# Set proper column names
df1.columns = df1.iloc[0]
df1 = df1[1:]
df1.reset_index(drop=True, inplace=True)

df2.columns = df2.iloc[0]
df2 = df2[1:]
df2.reset_index(drop=True, inplace=True)

In [107]:
# Non ho idea se la colonna Ebene (level) serva. 
# Spero di cuore che la colonna Code ci consenta di matchare i codici delle parti su Unity
cols_interst = [
"Code",
"Ebene",
"DIN-Code",
"Bezeichnung Italienisch\nmax. 60 Char"
]

df1 = df1[cols_interst]
df2 = df2[cols_interst]

# Remove last 2 rows from df1 since they are unnecessary
df1 = df1.loc[:2302]

In [108]:
df1.rename(columns={"Code": "component_code",
                   "Ebene" : "level",
                   "DIN-Code" : "DIN_code",
                   "Bezeichnung Italienisch\nmax. 60 Char" : "component_name"}, inplace=True)

df2.rename(columns={"Code": "component_code",
                   "Ebene" : "level",
                   "DIN-Code" : "DIN_code",
                   "Bezeichnung Italienisch\nmax. 60 Char" : "component_name"}, inplace=True)

In [109]:
# To remove the DIN_Code issue with NaN that is recognized ass part of the other DIN_Code column
df1.columns = [f"{col}_{i}" if df1.columns.tolist().count(col) > 1 else col for i, col in enumerate(df1.columns)]
df2.columns = [f"{col}_{i}" if df2.columns.tolist().count(col) > 1 else col for i, col in enumerate(df2.columns)]

df1.drop("DIN_code_3", axis=1, inplace=True)
df2.drop("DIN_code_3", axis=1, inplace=True)

df1.rename(columns={"DIN_code_2" : "DIN_code"}, inplace=True)
df2.rename(columns={"DIN_code_2" : "DIN_code"}, inplace=True)


In [110]:
df1.head()

,component_code,level,DIN_code,component_name
0,10,1,M-FA00,Alimentazione di energia
1,10_10,2,M-FB00,Alimentazione di energia da linea di contatto
2,10_10_10,3,M-FB13,Pantografo
3,10_10_10_10,4,M-FB13,Telaio di base
4,10_10_10_20,4,M-FB13,Isolatore


In [111]:
df2.head()

,component_code,level,DIN_code,component_name
0,10,1,M-FA00,Alimentazione di energia
1,10_10,2,M-FB00,Alimentazione di energia da linea di contatto
2,10_10_10,3,M-FB13,Pantografo
3,10_10_10_10,4,M-FB13,Telaio di base
4,10_10_10_20,4,M-FB13,Isolatore


In [112]:
df1.tail()

,component_code,level,DIN_code,component_name
2298,170_50_20_10,4,M-JD03,Bottiglie d’acqua
2299,170_50_20_20,4,M-JD03,Bombole di azoto
2300,170_50_20_30,4,M-JD03,Riscaldam. bottiglie d’acqua
2301,170_60,2,M-JB17,Prot. e sicurezza elettr. sist. segnal. e lott...
2302,170_70,2,M-JB17,Cablaggio elettr. sist. di segnalazione e lott...


In [113]:
df2.tail()

,component_code,level,DIN_code,component_name
2298,170_50_20_10,4,M-JD03,Bottiglie d’acqua
2299,170_50_20_20,4,M-JD03,Bombole di azoto
2300,170_50_20_30,4,M-JD03,Riscaldam. bottiglie d’acqua
2301,170_60,2,M-JB17,Prot. e sicurezza elettr. sist. segnal. e lott...
2302,170_70,2,M-JB17,Cablaggio elettr. sist. di segnalazione e lott...


In [114]:
# Print the differences between the 2 dataframes
differences = df1.compare(df2)

print("Differences between the DataFrames:")
print(differences)

Differences between the DataFrames:
     DIN_code        
         self   other
1807   M-LC03  M-LA01
1808   M-LC03  M-LA01


In [115]:
# I save both
df1.to_csv("data/interim/components1.csv", index=False)
df2.to_csv("data/interim/components2.csv", index=False)